In [4]:
from ladybug.epw import EPW

# load test data from a file
epw_file = (
    r"C:\Users\tgerrish\Documents\GitHub\target_dir\kvrrj\tests\assets\example.epw"
)
epw = EPW(epw_file)


In [271]:
import scipy
from kvrrj.wind import Wind
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from collections import defaultdict

w = Wind.from_epw(epw)

# create data
datetimes = [datetime(2021, 1, 1, 0, 0) + timedelta(hours=i) for i in range(8)]
ws = [1, 2, 3, 4, 5, 6, 7, 8]
wd = [0, 45, 90, 135, 180, 225, 270, 315]
other_data = [1, 2, 3, 4, 5, 6, 7, 8]
other_bins = [0, 2, 4, 6, 8]
# create wind object
w = Wind(
    wind_speeds=ws,
    wind_directions=wd,
    datetimes=datetimes,
    height_above_ground=10,
)

binned_data = w._bin_other_data(other_bins=other_bins)
binned_data

[(1, 2.0),
 (1, 2.0),
 (2.0, 4.0),
 (2.0, 4.0),
 (4.0, 6.0),
 (4.0, 6.0),
 (6.0, 8.0),
 (6.0, 8.0)]

In [ ]:


right = True
direction_data = w.wind_directions
other_data = w.wind_speeds

directions = 4
direction_bins = np.concat([[0], np.unique(w._direction_bins(directions)), [360]])
other_bins = np.linspace(min(other_data), max(other_data), 11)

if max(other_bins) < max(other_data):
    raise ValueError("The maximum value of the bins is less than the maximum value of the data")

# NOTE - data is binned in the form of [i, j) where the leftmost value is included and the rightmost value is excluded]
H, edge_x, edge_y, binnumber = scipy.stats.binned_statistic_2d(
    direction_data,
    other_data,
    None,
    bins=(direction_bins, other_bins),
    statistic="count",
    expand_binnumbers=True,
)

#combine the ends of H
H[0, :] = (H[-1, :] + H[0, :])
H = H[:-1, :]

# combine the ends of binnumber[0]
x_indices = np.where(binnumber[0] == max(binnumber[0]), 1, binnumber[0]) -1
y_indices = binnumber[1] - 1

# construct the bin labels
x_bins = [tuple(i) for i in w._direction_bins(directions)]
y_bins = [tuple(i) for i in np.lib.stride_tricks.sliding_window_view(other_bins, 2).tolist()]

# x_bins_str = [f"[{i}-{j})" for i, j in w._direction_bins(directions)]
# x_bins_str = [f"{i}" for i in w._direction_midpoints(directions)]
# y_bins_str = [f"[{i}-{j})" for i, j in np.lib.stride_tricks.sliding_window_view(other_bins, 2).round(3)]
# y_bins_str[-1] = y_bins_str[-1].replace(")", "]")
# y_bins_str

pd.concat([w.wd.reset_index(drop=True), pd.Series(x_indices), pd.Series([x_bins[i] for i in x_indices])], axis=1)

pd.concat(
    [
        w.ws.reset_index(drop=True),
        pd.Series(y_indices),
        pd.Series([y_bins[i] for i in y_indices]),
    ],
    axis=1,
)

,Wind Speed (m/s),0,1
0,7.7,4,"(7.0, 8.75)"
1,1.0,0,"(0.0, 1.75)"
2,2.6,1,"(1.75, 3.5)"
3,1.5,0,"(0.0, 1.75)"
4,2.1,1,"(1.75, 3.5)"
...,...,...,...
8755,6.7,3,"(5.25, 7.0)"
8756,4.1,2,"(3.5, 5.25)"
8757,4.6,2,"(3.5, 5.25)"
8758,5.1,2,"(3.5, 5.25)"


In [44]:
import pandas as pd
import numpy as np
directions = 3
right = True
boundaries = [(i.left - (360 / directions / 2)) % 360 for i in pd.interval_range(start=0, end=360, periods=directions).values]
boundaries.append(boundaries[0])
np.lib.stride_tricks.sliding_window_view(boundaries, 2, axis=0)

array([[300.,  60.],
       [ 60., 180.],
       [180., 300.]])

In [16]:
from kvrrj.solar import tilt_orientation_factor, EPW, azimuthal_radiation


epw = EPW(r"C:\Users\tgerrish\Documents\GitHub\target_dir\kvrrj\tests\assets\example.epw")
epw
tilt_orientation_factor(epw).shape

# azimuthal_radiation(epw)

100%|██████████| 9/9 [00:00<00:00, 3763.58it/s]


(37, 27)